In [9]:
import json
import csv

with open('/home/pj20/gode/data_process/valid_smiles_ids.json', 'r') as f:
    valid_smiles_ids = json.load(f)

with open('/home/pj20/gode/data_process/valid_smiles.csv') as f:
    reader = csv.reader(f)
    next(reader)  # skip header

    valid_smiles = []
    for line in reader:
        smiles = line[0]
        valid_smiles.append(line[0])

In [10]:
len(valid_smiles_ids), len(valid_smiles)

(41212, 41212)

In [11]:
smile2entid = {}

for i in range(len(valid_smiles)):
    smile2entid[valid_smiles[i]] = valid_smiles_ids[i]

In [12]:
import pickle

with open('/data/pj20/molkg_kge/transe/entity_embedding_1200.pkl', 'rb') as f:
    ent_emb = pickle.load(f)

In [13]:
ent_emb = ent_emb.cpu().detach().numpy()
len(ent_emb)

184819

In [14]:
ent_emb[-1]

array([-0.05708027, -0.01096982, -0.02211162, ...,  0.03796372,
       -0.00046855, -0.01706253], dtype=float32)

In [15]:
import numpy as np
from tqdm import tqdm

def emb_map(task):
    with open(f"./exampledata/finetune/{task}.csv") as f:
        reader = csv.reader(f)
        next(reader)  # skip header

        smiles = []
        for line in reader:
            smiles.append(line[0])


    pre_feature = features = np.load(f"./exampledata/finetune/{task}.npz")['features']
    
    kge_emb_ = []
    for i in tqdm(range(len(smiles))):
        smile = smiles[i]
        if smile in smile2entid.keys():
            id_ = smile2entid[smile]
            emb = ent_emb[id_]
            kge_emb_.append(emb)
        else:
            emb = np.zeros(len(ent_emb[0]))
            kge_emb_.append(emb)
    
    kge_emb_ = np.array(kge_emb_)

    post_feature = np.concatenate((pre_feature, kge_emb_), axis=1)

    # np.save(f"./exampledata/finetune/{task}_kge.npy", kge_emb_)
    np.save(f"./exampledata/finetune/{task}_fg_kge.npy", post_feature)
    
    return kge_emb_

In [16]:
tasks = ['bace', 'bbbp', 'clintox', 'esol', 'freesolv', 'lipo', 'qm7', 'qm8', 'sider', 'tox21', 'toxcast']

for task in tasks:
    emb_map(task=task)

100%|██████████| 8576/8576 [00:00<00:00, 623208.55it/s]


In [53]:
len(np.concatenate((np.load('./bace.npz')['features'], np.load('./bace_kge.npy')), axis=1)[0])


1400